In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import os
from os.path import exists
import time
import sys
sys.path.insert(1, '../../framework')
from racetrack import *
rt = RACETrack()
ofv = rt.ontologyFrameworkInstance()

In [ ]:
#
# Email File First // file has utf8 issues :(
#
_bin_ = open('../../../data/2014_vast/MC1/email headers.csv', 'rb').read()
print(len(_bin_))
as_str = []
for i in range(len(_bin_)):
    c = chr(_bin_[i])
    if (c >= 'a' and c <= 'z') or (c >= 'A' and c <= 'Z') or (c >= '0' and c <= '9') or \
       c == ' ' or c == '.' or c == ',' or c == '_' or \
       c == '-' or c == '!' or c == '@' or c == '"' or \
       c == ':' or c == '\t' or c == '\n' or c == '\r' or \
       c == '/' or c == '?' or c == ')' or c == '(' or \
       c == "'":
        as_str.append(c)
    elif ord(c) == 146:
        as_str.append("'")
    else:
        print('"'+c+'"', ord(c))
_str_ = ''.join(as_str)
print(len(_str_))
open('../../../data/2014_vast/MC1/email_headers_fixed.csv', 'wt').write(_str_)
df_email = pl.read_csv('../../../data/2014_vast/MC1/email_headers_fixed.csv')
df_email.sample(3)

In [ ]:
#
# Ontology Based On:
# https://www.semanticdesktop.org/ontologies/2007/03/22/nmo/
#
for i in range(len(df_email)):
    _id_              = ofv.createId('nmo:Email')
    _fm_              = df_email['From'][i].strip()
    _fm_id_           = ofv.resolveUniqIdAndUpdateLookups(_fm_, 'nmo:EmailAddress', 'uniq', 'sbj,obj')
    _timestamp_       = df_email['Date'][i]
    _ts_as_int_       = ofv.resolveUniqIdAndUpdateLookups(_timestamp_, 'xsd:dateTime', 'dttm', 'obj')
    _subject_         = df_email['Subject'][i].strip()
    _subject_literal_ = ofv.resolveUniqIdAndUpdateLookups(_subject_, 'nmo:MessageHeader', 'cont', 'obj')
    ofv.bufferTripleToAddLater(_id_, 'nmo:emailFrom',   _fm_id_)
    ofv.bufferTripleToAddLater(_id_, 'nmo:sentDate',    _ts_as_int_)
    ofv.bufferTripleToAddLater(_id_, 'nmo:headerValue', _subject_literal_)
    _to_              = df_email['To'][i]
    for _to_actual_ in _to_.split(','):
        _to_actual_ = _to_actual_.strip()
        _to_actual_id_ = ofv.resolveUniqIdAndUpdateLookups(_to_actual_, 'nco:EmailAddress', 'uniq', 'sbj,obj')
        ofv.bufferTripleToAddLater(_id_, 'nmo:emailTo', _to_actual_id_)
ofv.appendBufferedTriplesAndClearBuffer()
len(ofv.df_triples)

In [ ]:
df_employee_recs =pl.read_excel('../../../data/2014_vast/MC1/EmployeeRecords.xlsx')

In [ ]:
def fixDate(_date_):
    if type(_date_) != str: return _date_
    _split_ = _date_.split('-')
    _year_, _month_, _day_ = int(_split_[2]), int(_split_[0]), int(_split_[1])
    if _year_ < 30: _year_ += 2000
    else:           _year_ += 1900
    _date_str_ = f'{_year_}-{_month_:02}-{_day_:02}'
    return _date_str_

#
# https://w3id.org/MON/person.owl
# https://www.w3.org/TR/vocab-org/
#
#
for i in range(len(df_employee_recs)):
    _id_ = ofv.createId('mon:Person')
    _last_, _first_, _dob_, _pob_, _gender_ = df_employee_recs['LastName'][i], df_employee_recs['FirstName'][i], df_employee_recs['BirthDate'][i], df_employee_recs['BirthCountry'][i], df_employee_recs['Gender'][i]
    _last_id_   = ofv.resolveUniqIdAndUpdateLookups(_last_,         'xsd:string', 'ambi', 'obj')
    _first_id_  = ofv.resolveUniqIdAndUpdateLookups(_first_,        'xsd:string', 'ambi', 'obj')
    _pob_id_    = ofv.resolveUniqIdAndUpdateLookups(_pob_,          'xsd:string', 'uniq', 'obj')
    _dob_id_    = ofv.resolveUniqIdAndUpdateLookups(fixDate(_dob_), 'xsd:date',   'date', 'obj')
    _gender_id_ = ofv.resolveUniqIdAndUpdateLookups(_gender_,       'mon:Sex',    'cata', 'obj')

    ofv.bufferTripleToAddLater(_id_, 'mon:hasBirthDate',  _dob_id_)
    ofv.bufferTripleToAddLater(_id_, 'mon:hasBirthPlace', _pob_id_)
    ofv.bufferTripleToAddLater(_id_, 'mon:hasGender',     _gender_id_)
    ofv.bufferTripleToAddLater(_id_, 'mon:lastName',      _last_id_)
    ofv.bufferTripleToAddLater(_id_, 'mon:firstName',     _first_id_)

    _employment_department_  = df_employee_recs['CurrentEmploymentType'][i]
    _employment_title_       = df_employee_recs['CurrentEmploymentTitle'][i]
    _employment_start_date_  = df_employee_recs['CurrentEmploymentStartDate'][i]

    _employment_id_          = ofv.createId('org:Employment')
    ofv.bufferTripleToAddLater(_id_, 'hasEmployment',  _employment_id_)

    _employment_department_id_    = ofv.resolveUniqIdAndUpdateLookups(_employment_department_,          'org:Organization',      'uniq',     'sbj,obj')
    _employment_title_id_         = ofv.resolveUniqIdAndUpdateLookups(_employment_title_,               'org:EmploymentTitle',   'ambi',     'sbj,obj')
    _employment_start_date_id_    = ofv.resolveUniqIdAndUpdateLookups(fixDate(_employment_start_date_), 'xsd:date',              'date',     'obj')

    ofv.bufferTripleToAddLater(_employment_id_, 'employedBy',          _employment_department_id_)
    ofv.bufferTripleToAddLater(_employment_id_, 'hasEmploymentTitle',  _employment_title_id_)
    ofv.bufferTripleToAddLater(_employment_id_, 'hasStartDate',        _employment_start_date_id_)

    _email_address_          = df_employee_recs['EmailAddress'][i].strip()
    _email_address_id_       = ofv.resolveUniqIdAndUpdateLookups(_email_address_, 'nmo:EmailAddress', 'uniq', 'sbj,obj')

    ofv.bufferTripleToAddLater(_id_, 'nmo:hasEmailAddress', _email_address_id_)

ofv.appendBufferedTriplesAndClearBuffer()
len(ofv.df_triples)

In [ ]:
ln_params = {'relationships':[('sbj','obj')], 'link_opacity':0.1, 'node_labels':ofv.nodeLabels()}
_rtg_ = rt.interactiveGraphLayout(ofv.df_triples, ln_params, w=800, h=600)
if exists('../../../data/2014_vast/layout_example.csv'): _rtg_.loadLayout('../../../data/2014_vast/layout_example.csv')
_rtg_

In [ ]:
#rt.link(ofv.df_triples, **ln_params)
#_rtg_.selectEntities('Morlun',method='substring')
#_rtg_.selectedEntities()
#_rtg_.saveLayout('../../../data/2014_vast/layout_example.csv')

In [ ]:
#!pip install python-docx
import docx
_base_resumes_dir_ = '../../../data/2014_vast/MC1/resumes/'
for _file_ in os.listdir(_base_resumes_dir_):
    _docx_ = docx.Document(_base_resumes_dir_+_file_)
    for _para_ in _docx_.paragraphs:
        pass
        #print(_para_.text)
    #print('---\n---\n---')